<a href="https://colab.research.google.com/github/marcelo-guimaraes/Data-Science/blob/master/SpamClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This Project is a Spam Classifier and it'ss part of the Text Mining course at the University of Michigan. Aims to create features for text data and use a Machine Learning model using the NLTK library to predict the chances of an email being Spam or not

### First I'll start by importing my libraries and data

In [107]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('/content/drive/My Drive/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)

print('File size: {} lines\n'.format(len(spam_data)))
spam_data.head(10)

File size: 5572 lines



,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [0]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [109]:
print("Percentage of emails that are spam: {}".format(spam_data['target'].mean() * 100))

Percentage of emails that are spam: 13.406317300789663


### The metrics used for evaluating the model was selected by this article of [Machine Learning Mastery](https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/) 

### Question 1

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [110]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from imblearn.metrics import geometric_mean_score

vet = CountVectorizer().fit(X_train)
train_vet = vet.transform(X_train)

clf = MultinomialNB(alpha = 0.1).fit(train_vet, y_train)

y_pred = clf.predict(vet.transform(X_test))

print("AUC Score: {}".format(roc_auc_score(y_test, y_pred)),
      "\nRecall SCORE: {}".format(recall_score(y_pred, y_test)),
      "\nPrecision SCORE: {}".format(precision_score(y_pred, y_test)),
      "\nG-Mean SCORE: {}".format(geometric_mean_score(y_pred, y_test)))

AUC Score: 0.9720812182741116 
Recall SCORE: 1.0 
Precision SCORE: 0.9441624365482234 
G-Mean SCORE: 0.9954328181465828


### Question 2

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

vectorizer = TfidfVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)

feature_names = vectorizer.get_feature_names()
idfs = vectorizer.idf_
names_idfs = list(zip(feature_names, idfs))

smallest = sorted(names_idfs, key=operator.itemgetter(1))[:20]
smallest = pd.Series([features[1] for features in smallest], index=[features[0] for features in smallest])

largest = sorted(names_idfs, key=operator.itemgetter(1), reverse=True)[:20]
largest = sorted(largest, key=operator.itemgetter(0))
largest = pd.Series([features[1] for features in largest], index=[features[0] for features in largest])

(smallest, largest)

(to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64, 000pes         8.644919
 0089           8.644919
 0121           8.644919
 01223585236    8.644919
 0125698789     8.644919
 02072069400    8.644919
 02073162414    8.644919
 02085076972    8.644919
 021            8.644919
 0430           8.644919
 07008009200    8.644919
 07099833605    8.644919
 07123456789    8.644919
 0721072        8.644919
 07753741225    8.644919
 077xxx         8.644919
 078            8.644919
 07808247860    8.644919
 07808726822    8.644919
 078498         8.644919
 dtype: float64)

### Question 3

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [112]:
vet = TfidfVectorizer(min_df=3).fit(X_train)
train_vet = vet.transform(X_train)

clf = MultinomialNB(alpha=0.1).fit(train_vet, y_train)

y_pred = clf.predict(vet.transform(X_test))
        
print("AUC Score: {}".format(roc_auc_score(y_test, y_pred)),
      "\nRecall SCORE: {}".format(recall_score(y_pred, y_test)),
      "\nPrecision SCORE: {}".format(precision_score(y_pred, y_test)),
      "\nG-Mean SCORE: {}".format(geometric_mean_score(y_pred, y_test)))

AUC Score: 0.9416243654822335 
Recall SCORE: 1.0 
Precision SCORE: 0.883248730964467 
G-Mean SCORE: 0.9905211130872972


#### Well, as we can see. Using a min_df of 3 worsened the model

### Question 4

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [113]:
spam_data['length'] = spam_data['text'].apply(lambda x:len(x))

print("Average Length of not Spam Documents: {} characters".format(int(np.mean(spam_data['length'][spam_data['target'] == 0]))), 
      "\nAverage Length of Spam Documents: {} characters".format(int(np.mean(spam_data['length'][spam_data['target'] == 1]))))

Average Length of not Spam Documents: 71 characters 
Average Length of Spam Documents: 138 characters


In [0]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 5

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [115]:
from sklearn.svm import SVC



vectorizer = TfidfVectorizer(min_df=5)

train_vet = vectorizer.fit_transform(X_train)
train_vet_len = add_feature(train_vet, X_train.str.len())

teste = vectorizer.transform(X_test) 
teste_len = add_feature(teste, X_test.str.len())

clf = SVC(C=10000).fit(train_vet_len, y_train)

y_pred = clf.predict(teste_len)
  
   
print("AUC Score: {}".format(roc_auc_score(y_test, y_pred)),
      "\nRecall SCORE: {}".format(recall_score(y_pred, y_test)),
      "\nPrecision SCORE: {}".format(precision_score(y_pred, y_test)),
      "\nG-Mean SCORE: {}".format(geometric_mean_score(y_pred, y_test)))

AUC Score: 0.9661689557407943 
Recall SCORE: 0.989247311827957 
Precision SCORE: 0.934010152284264 
G-Mean SCORE: 0.9892384031812388


### Question 6

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [116]:
spam_data['digits'] = spam_data['text'].apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
    
(spam_data['digits'][spam_data['target'] == 0].mean(), spam_data['digits'][spam_data['target'] == 1].mean())

(0.2992746113989637, 15.759036144578314)

### Question 7

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [117]:
from sklearn.linear_model import LogisticRegression


    
vectorizer = TfidfVectorizer(min_df=5, ngram_range=[1,3])

train_vet = vectorizer.fit_transform(X_train)
X_train_transformed = add_feature(train_vet, [X_train.str.len(),
                                              X_train.apply(lambda x:len(''.join([a for a in x if a.isdigit()])))])

test_vet = vectorizer.transform(X_test)
X_test_transformed = add_feature(test_vet,[X_test.str.len(),
                                           X_test.apply(lambda x:len(''.join(a for a in x if a.isdigit())))])

clf = LogisticRegression(C=100,max_iter = 2000).fit(X_train_transformed, y_train)

y_pred = clf.predict(X_test_transformed)

    
print("AUC Score: {}".format(roc_auc_score(y_test, y_pred)),
      "\nRecall SCORE: {}".format(recall_score(y_pred, y_test)),
      "\nPrecision SCORE: {}".format(precision_score(y_pred, y_test)),
      "\nG-Mean SCORE: {}".format(geometric_mean_score(y_pred, y_test)))

AUC Score: 0.9678709064054463 
Recall SCORE: 0.9788359788359788 
Precision SCORE: 0.9390862944162437 
G-Mean SCORE: 0.9844186804989808


### Question 8

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [118]:
spam = spam_data['text'][spam_data['target'] == 1].str.findall(r'(\W)').str.len().mean()
not_spam = spam_data['text'][spam_data['target'] == 0].str.findall(r'(\W)').str.len().mean()
    
print("Avg N° of non-word characters in non Spam Emails: {}".format(int(not_spam)), 
      "\nAvg N° of non-word characters inSpam Emails: {}".format(int(spam)))

Avg N° of non-word characters in non Spam Emails: 17 
Avg N° of non-word characters inSpam Emails: 29


### Question 9

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [119]:
vectorizer = CountVectorizer(min_df=5, ngram_range = [2,5], analyzer = 'char_wb').fit(X_train)

X_train_vet = vectorizer.transform(X_train)
X_test_vet = vectorizer.transform(X_test)

X_train_added = add_feature(X_train_vet, [X_train.str.len(),
                                          X_train.apply(lambda x:len(''.join([a for a in x if a.isdigit()]))),
                                          X_train.str.findall(r'(\W)').str.len()])

X_test_added = add_feature(X_test_vet, [X_test.str.len(),
                                        X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                        X_test.str.findall(r'(\W)').str.len()])

clf = LogisticRegression(C=100,max_iter = 2000)

clf.fit(X_train_added, y_train)

y_pred = clf.predict(X_test_added)

auc = roc_auc_score(y_test, y_pred)

print("AUC Score: {}".format(auc),
      "\nRecall SCORE: {}".format(recall_score(y_pred, y_test)),
      "\nPrecision SCORE: {}".format(precision_score(y_pred, y_test)),
      "\nG-Mean SCORE: {}".format(geometric_mean_score(y_pred, y_test)))

AUC Score: 0.9780231906694056 
Recall SCORE: 0.9792746113989638 
Precision SCORE: 0.9593908629441624 
G-Mean SCORE: 0.9862789230180463


In [0]:
feature_names = np.array(vectorizer.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
sorted_coef_index = clf.coef_[0].argsort()
smallest = feature_names[sorted_coef_index[:10]]
largest = feature_names[sorted_coef_index[:-11:-1]]

In [121]:
(auc, list(smallest), list(largest)) 

(0.9780231906694056,
 [' i', 'ca', '..', '. ', 'pe', ' go', ' m', 'if', 'us', 'go'],
 ['digit_count', 'ia', ' r', 'xt', 'ne', 'co', ' ba', ' x', 'ian ', '46'])